In [1]:
from pysparkling import *
hc = H2OContext(sc).start()

H2O cluster uptime:,11 seconds 856 milliseconds
H2O cluster version:,3.2.0.3
H2O cluster name:,sparkling-water-kuba
H2O cluster total nodes:,1
H2O cluster total memory:,491.5 MB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,192.168.0.184
H2O Connection port:,54325


In [2]:
# Import cluster local file - we need to add files via spark import files  -FIXME
import h2o
#from h2o.h2o import _locate # private function. used to find files within h2o project directory.
def _locate(s): 
    return "../../../" + s 
   

f_weather = h2o.import_file(_locate("examples/smalldata/chicagoAllWeather.csv"))
f_census = h2o.import_file(_locate("examples/smalldata/chicagoCensus.csv"))
f_crimes = h2o.import_file(_locate("examples/smalldata/chicagoCrimes10k.csv"))


Parse Progress: [##################################################] 100%
Imported ../../../examples/smalldata/chicagoAllWeather.csv. Parsed 5,162 rows and 7 cols

Parse Progress: [##################################################] 100%
Imported ../../../examples/smalldata/chicagoCensus.csv. Parsed 79 rows and 9 cols

Parse Progress: [##################################################] 100%
Imported ../../../examples/smalldata/chicagoCrimes10k.csv. Parsed 9,999 rows and 22 cols


In [3]:
f_weather

H2OFrame with 5162 rows and 7 columns: 
      date  month  day  year  maxTemp  meanTemp  minTemp
0   1/1/01      1    1  2001       23        14        6
1   1/2/01      1    2  2001       18        12        6
2   1/3/01      1    3  2001       28        18        8
3   1/4/01      1    4  2001       30        24       19
4   1/5/01      1    5  2001       36        30       21
5   1/6/01      1    6  2001       33        26       19
6   1/7/01      1    7  2001       34        28       21
7   1/8/01      1    8  2001       26        20       14
8   1/9/01      1    9  2001       23        16       10
9  1/10/01      1   10  2001       34        26       19


In [4]:
# Transform weather table
## Remove 1st column (date)
f_weather = f_weather[1:]

In [5]:
# Transform census table
## Remove all spaces from column names (causing problems in Spark SQL)
col_names = map(lambda s: s.strip().replace(' ', '_'), f_census.col_names)

## Update column names in the table
#f_weather.setNames(col_names)
f_census.setNames(col_names)

H2OFrame with 79 rows and 9 columns: 
   Community_Area_Number  COMMUNITY_AREA_NAME  PERCENT_OF_HOUSING_CROWDED  \
0                    NaN  COMMUNITY AREA NAME                         NaN   
1                      1          Rogers Park                         7.7   
2                      2           West Ridge                         7.8   
3                      3               Uptown                         3.8   
4                      4       Lincoln Square                         3.4   
5                      5         North Center                         0.3   
6                      6            Lake View                         1.1   
7                      7         Lincoln Park                         0.8   
8                      8      Near North Side                         1.9   
9                      9          Edison Park                         1.1   

   PERCENT_HOUSEHOLDS_BELOW_POVERTY  PERCENT_AGED_16__UNEMPLOYED  \
0                               NaN           

In [6]:
# Transform crimes table

## Drop useless columns
f_crimes = f_crimes[2:]

## Replace ' ' by '_' in column names
col_names = map(lambda s: s.replace(' ', '_'), f_crimes.col_names)
f_crimes.setNames(col_names)

## Refine date column
def refine_date_col(data, col, pattern):
    data[col]         = data[col].as_date(pattern)
    data["Day"]       = data[col].day()
    data["Month"]     = data[col].month()
    data["Year"]      = data[col].year()
    data["WeekNum"]   = data[col].week()
    data["WeekDay"]   = data[col].dayOfWeek()
    data["HourOfDay"] = data[col].hour()
    
    data.describe()  # HACK: Force evaluation before ifelse and cut. See PUBDEV-1425.
    
    # Create weekend and season cols
    # Spring = Mar, Apr, May. Summer = Jun, Jul, Aug. Autumn = Sep, Oct. Winter = Nov, Dec, Jan, Feb.
    # data["Weekend"]   = [1 if x in ("Sun", "Sat") else 0 for x in data["WeekDay"]]
    data["Weekend"] = h2o.ifelse(data["WeekDay"] == "Sun" or data["WeekDay"] == "Sat", 1, 0)[0]
    data["Season"] = data["Month"].cut([0, 2, 5, 7, 10, 12], ["Winter", "Spring", "Summer", "Autumn", "Winter"])
    
refine_date_col(f_crimes, "Date", "%m/%d/%Y %I:%M:%S %p")
f_crimes = f_crimes.drop("Date")
f_crimes.describe()

Rows: 9,999 Cols: 25
  chunk_type                 chunk_name  count  count_percentage        size  \
0        C0L          Constant Integers     58         12.888889      4.5 KB   
1        CBS                       Bits     37          8.222222      5.1 KB   
2         C1            1-Byte Integers     41          9.111111     25.0 KB   
3        C1N  1-Byte Integers (w/o NAs)    152         33.777775     92.5 KB   
4         C2            2-Byte Integers     72         16.000000     82.9 KB   
5        C2S           2-Byte Fractions      2          0.444444      2.4 KB   
6         C4            4-Byte Integers     36          8.000000     80.5 KB   
7         C8            64-bit Integers     16          3.555556     70.4 KB   
8        C8D               64-bit Reals     36          8.000000    158.6 KB   

   size_percentage  
0         0.868301  
1         0.968605  
2         4.782766  
3        17.728870  
4        15.885414  
5         0.452115  
6        15.427311  
7        1

,Date,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Updated_On,Latitude,Longitude,Location,Day,Month,WeekNum,WeekDay,HourOfDay
type,int,enum,int,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,enum,real,real,enum,int,int,int,enum,int
mins,1.42199823e+12,0.0,110.0,0.0,0.0,0.0,0.0,0.0,111.0,1.0,1.0,1.0,2.0,1100317.0,1814255.0,2015.0,0.0,41.64507243,-87.906463888,0.0,1.0,1.0,4.0,0.0,0.0
maxs,1.42343542e+12,6517.0,5131.0,26.0,198.0,90.0,1.0,1.0,2535.0,25.0,50.0,77.0,26.0,1205069.0,1951533.0,2015.0,32.0,42.022646183,-87.524773286,8603.0,31.0,2.0,6.0,6.0,23.0
sigma,433879245.189,1915.88517194,927.751435583,9.16241735944,60.1059382029,25.5963972463,0.455083515588,0.35934414686,695.76029875,6.94547493301,13.6495661144,21.2748762223,7.57423857911,16496.4493681,31274.0163199,0.0,10.0824464345,0.0860186579359,0.0600357970653,2469.64729385,11.1801043358,0.493492406787,0.738929830409,1.93284056432,6.47321735807
zero_count,0,3,0,11,933,19,7071,8476,0,0,0,0,0,0,0,0,603,0,0,1,0,0,0,1038,374
missing_count,0,0,419,0,0,6,0,0,0,162,0,0,2557,162,162,0,0,162,162,162,0,0,0,0,0


Rows: 9,999 Cols: 26
  chunk_type                 chunk_name  count  count_percentage        size  \
0        C0L          Constant Integers     89         19.017094      7.0 KB   
1        CBS                       Bits     42          8.974360      5.7 KB   
2         C1            1-Byte Integers     41          8.760684     25.0 KB   
3        C1N  1-Byte Integers (w/o NAs)    152         32.478634     92.5 KB   
4         C2            2-Byte Integers     72         15.384616     82.9 KB   
5         C4            4-Byte Integers     36          7.692308     80.5 KB   
6        C8D               64-bit Reals     36          7.692308    158.6 KB   

   size_percentage  
0         1.537618  
1         1.269182  
2         5.519443  
3        20.459600  
4        18.332203  
5        17.803540  
6        35.078415  
                              size  number_of_rows  \
0  192.168.0.184:54321    452.2 KB            9999   
1                 mean    452.2 KB            9999   
2       

,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Updated_On,Latitude,Longitude,Location,Day,Month,WeekNum,WeekDay,HourOfDay,Weekend,Season
type,enum,int,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,enum,real,real,enum,int,int,int,enum,int,int,enum
mins,0.0,110.0,0.0,0.0,0.0,0.0,0.0,111.0,1.0,1.0,1.0,2.0,1100317.0,1814255.0,2015.0,0.0,41.64507243,-87.906463888,0.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0
maxs,6517.0,5131.0,26.0,198.0,90.0,1.0,1.0,2535.0,25.0,50.0,77.0,26.0,1205069.0,1951533.0,2015.0,32.0,42.022646183,-87.524773286,8603.0,31.0,2.0,6.0,6.0,23.0,1.0,0.0
sigma,1915.88517194,927.751435583,9.16241735944,60.1059382029,25.5963972463,0.455083515588,0.35934414686,695.76029875,6.94547493301,13.6495661144,21.2748762223,7.57423857911,16496.4493681,31274.0163199,0.0,10.0824464345,0.0860186579359,0.0600357970653,2469.64729385,11.1801043358,0.493492406787,0.738929830409,1.93284056432,6.47321735807,0.365802434041,0.0
zero_count,3,0,11,933,19,7071,8476,0,0,0,0,0,0,0,0,603,0,0,1,0,0,0,1038,374,8408,9999
missing_count,0,419,0,0,6,0,0,0,162,0,0,2557,162,162,0,0,162,162,162,0,0,0,0,0,0,0


In [7]:
# Expose H2O frames as Spark DataFrame
f_weather._id

df_weather = hc.as_data_frame(f_weather)
df_census = hc.as_data_frame(f_census)
df_crimes = hc.as_data_frame(f_crimes)

In [8]:
df_weather.show()

+-----+---+----+-------+--------+-------+
|month|day|year|maxTemp|meanTemp|minTemp|
+-----+---+----+-------+--------+-------+
|    1|  1|2001|     23|      14|      6|
|    1|  2|2001|     18|      12|      6|
|    1|  3|2001|     28|      18|      8|
|    1|  4|2001|     30|      24|     19|
|    1|  5|2001|     36|      30|     21|
|    1|  6|2001|     33|      26|     19|
|    1|  7|2001|     34|      28|     21|
|    1|  8|2001|     26|      20|     14|
|    1|  9|2001|     23|      16|     10|
|    1| 10|2001|     34|      26|     19|
|    1| 11|2001|     39|      28|     18|
|    1| 12|2001|     37|      31|     25|
|    1| 13|2001|     35|      34|     33|
|    1| 14|2001|     36|      34|     32|
|    1| 15|2001|     35|      32|     30|
|    1| 16|2001|     30|      28|     26|
|    1| 17|2001|     26|      22|     19|
|    1| 18|2001|     30|      24|     19|
|    1| 19|2001|     27|      22|     17|
|    1| 20|2001|     24|      18|     10|
+-----+---+----+-------+--------+-

In [9]:
# Use Spark SQL to join datasets

## Register DataFrames as tables in SQL context
sqlContext.registerDataFrameAsTable(df_weather, "chicagoWeather")
sqlContext.registerDataFrameAsTable(df_census, "chicagoCensus")
sqlContext.registerDataFrameAsTable(df_crimes, "chicagoCrime")


crimeWithWeather = sqlContext.sql("""SELECT
a.Year, a.Month, a.Day, a.WeekNum, a.HourOfDay, a.Weekend, a.Season, a.WeekDay,
a.IUCR, a.Primary_Type, a.Location_Description, a.Community_Area, a.District,
a.Arrest, a.Domestic, a.Beat, a.Ward, a.FBI_Code,
b.minTemp, b.maxTemp, b.meanTemp,
c.PERCENT_AGED_UNDER_18_OR_OVER_64, c.PER_CAPITA_INCOME, c.HARDSHIP_INDEX,
c.PERCENT_OF_HOUSING_CROWDED, c.PERCENT_HOUSEHOLDS_BELOW_POVERTY,
c.PERCENT_AGED_16__UNEMPLOYED, c.PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA
FROM chicagoCrime a
JOIN chicagoWeather b
ON a.Year = b.year AND a.Month = b.month AND a.Day = b.day
JOIN chicagoCensus c
ON a.Community_Area = c.Community_Area_Number""")

In [10]:
crimeWithWeather.show()

+----+-----+---+-------+---------+-------+------+-------+----+--------------------+--------------------+--------------+--------+------+--------+----+----+--------+-------+-------+--------+--------------------------------+-----------------+--------------+--------------------------+--------------------------------+---------------------------+--------------------------------------------+
|Year|Month|Day|WeekNum|HourOfDay|Weekend|Season|WeekDay|IUCR|        Primary_Type|Location_Description|Community_Area|District|Arrest|Domestic|Beat|Ward|FBI_Code|minTemp|maxTemp|meanTemp|PERCENT_AGED_UNDER_18_OR_OVER_64|PER_CAPITA_INCOME|HARDSHIP_INDEX|PERCENT_OF_HOUSING_CROWDED|PERCENT_HOUSEHOLDS_BELOW_POVERTY|PERCENT_AGED_16__UNEMPLOYED|PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA|
+----+-----+---+-------+---------+-------+------+-------+----+--------------------+--------------------+--------------+--------+------+--------+----+----+--------+-------+-------+--------+--------------------------------+---

In [11]:
# Split final data table
crimeWithWeatherHF = hc.as_h2o_frame(crimeWithWeather)
ratios = ["0.8", "0.2"]
frs = crimeWithWeatherHF.split_frame(ratios)
train = frs[0]
test = frs[1]


Split Frame Progress: [##################################################] 100%


In [95]:
#Build GBM model and collect model metrics
from h2o.model import ModelBase
gbmModel = h2o.gbm(x=train,y=train["Arrest"],validation_x=test,validation_y=test["Arrest"],
                   distribution="bernoulli", ntrees=10, max_depth=6)

# Model has methods to obtain particular metrics, but not all of them. We have to use ModelBase to get all metrics
metricsGBM = ModelBase._get_metrics(gbmModel, train=True, valid=True, xval=False)
trainMetricsGBM = metricsGBM["train"]
testMetricsGBM = metricsGBM["valid"]

#Print Scores of GBM
print("""
    Model performance (GBM):
          |    train AUC = """+str(trainMetricsGBM.auc())+"""
          |    test  AUC = """+str(testMetricsGBM.auc())+"""
          """)



gbm Model Build Progress: [##################################################] 100%

    Model performance (GBM):
          |    train AUC = 0.919733211547
          |    test  AUC = 0.939982096354
          


In [97]:
#Build Deep Learning model and collect model metrics
from h2o.model import ModelBase
from h2o.model.metrics_base import MetricsBase
glmModel = h2o.glm(x=train,y=train["Arrest"],validation_x=test,validation_y=test["Arrest"])

# Model has methods to obtain particular metrics, but not all of them. We have to use ModelBase to get all metrics
metricsGLM = ModelBase._get_metrics(glmModel, train=True, valid=True, xval=False)
trainMetricsGLM = metricsGLM["train"]
testMetricsGLM = metricsGLM["valid"]

#Print Scores of GLM
print("""
    Model performance (GLM):
          |    train AUC = """+str(trainMetricsGBM.auc())+"""
          |    test  AUC = """+str(testMetricsGBM.auc())+"""
          """)


glm Model Build Progress: [##################################################] 100%

    Model performance (GLM):
          |    train AUC = 0.919733211547
          |    test  AUC = 0.939982096354
          


In [204]:
# create Crime class
from datetime import datetime
from pytz import timezone

def get_season(dt):
    if (dt >= 3 and dt <= 5):
        return "Spring"
    elif (dt >= 6 and dt <= 8):
        return "Summer"
    elif (dt >= 9 and dt <= 10):
        return "Autumn"
    else:       
        return "Winter"

def crime(date,
        iucr,
        primaryType,
        locationDescr,
        domestic,
        beat,
        district,
        ward,
        communityArea,
        fbiCode,
        minTemp = 77777,
        maxTemp = 77777,
        meanTemp = 77777,
        datePattern = "%d/%m/%Y %I:%M:%S %p",
        dateTimeZone = "Etc/UTC"):

    dt = datetime.strptime("02/08/2015 11:43:58 PM",'%d/%m/%Y %I:%M:%S %p')
    dt.replace(tzinfo=timezone("Etc/UTC"))

    crime = {}
    crime["Year"] = dt.year
    crime["Month"] = dt.month
    crime["Day"] = dt.day
    crime["WeekNum"] = dt.isocalendar()[1]
    crime["HourOfDay"] = dt.hour
    crime["Weekend"] = 1 if dt.weekday() == 5 or dt.weekday() == 6 else 0
    crime["Season"] = get_season(dt.month)
    crime["WeekDay"] = dt.strftime('%a')  #gets the day of week in short format - Mon, Tue ...
    crime["IUCR"] = iucr
    crime["Primary_Type"] = primaryType
    crime["Location_Description"] = locationDescr
    crime["Domestic"] = True if domestic else False
    crime["Beat"] = beat
    crime["District"] = district
    crime["Ward"] = ward
    crime["Community_Area"] = communityArea
    crime["FBI_Code"] = fbiCode
    crime["minTemp"] = minTemp
    crime["maxTemp"] = maxTemp
    crime["meanTemp"] = meanTemp
    return crime

In [205]:
# Create crimes examples
crime_examples = [
  crime("02/08/2015 11:43:58 PM", 1811, "NARCOTICS", "STREET",False, 422, 4, 7, 46, 18),
  crime("02/08/2015 11:00:39 PM", 1150, "DECEPTIVE PRACTICE", "RESIDENCE",False, 923, 9, 14, 63, 11)]


In [219]:
#For given crime and model returns probability of crime.
def score_event(crime, model, censusTable):
    srdd = sqlContext.createDataFrame([crime_examples[0]])
    # Join table with census data
    df_row = df_census.join(srdd).where("Community_Area = Community_Area_Number")  
    row = hc.as_h2o_frame(df_row)
    
    #TODO: finish scoring
    return 1.0

for crime in crime_examples:
    arrestProbGBM = 100*score_event(crime, gbmModel, df_census)
    arrestProbGLM = 100*score_event(crime, gbmModel, df_census)
    print("""
       |Crime: """+str(crime)+"""
       |  Probability of arrest best on DeepLearning: """+str(arrestProbGLM)+"""
       |  Probability of arrest best on GBM: """+str(arrestProbGBM)+"""
        """)


       |Crime: {'meanTemp': 77777, 'maxTemp': 77777, 'Primary_Type': 'NARCOTICS', 'District': 4, 'minTemp': 77777, 'Beat': 422, 'Season': 'Summer', 'Community_Area': 46, 'Domestic': False, 'IUCR': 1811, 'Month': 8, 'Location_Description': 'STREET', 'WeekNum': 31, 'WeekDay': 'Sun', 'Year': 2015, 'HourOfDay': 23, 'Ward': 7, 'FBI_Code': 18, 'Weekend': 1, 'Day': 2}
       |  Probability of arrest best on DeepLearning: 100.0
       |  Probability of arrest best on GBM: 100.0
        

       |Crime: {'meanTemp': 77777, 'maxTemp': 77777, 'Primary_Type': 'DECEPTIVE PRACTICE', 'District': 9, 'minTemp': 77777, 'Beat': 923, 'Season': 'Summer', 'Community_Area': 63, 'Domestic': False, 'IUCR': 1150, 'Month': 8, 'Location_Description': 'RESIDENCE', 'WeekNum': 31, 'WeekDay': 'Sun', 'Year': 2015, 'HourOfDay': 23, 'Ward': 14, 'FBI_Code': 11, 'Weekend': 1, 'Day': 2}
       |  Probability of arrest best on DeepLearning: 100.0
       |  Probability of arrest best on GBM: 100.0
        


AttributeError: 'H2OBinomialModel' object has no attribute 'score'